# ML.Net - Taxi Prediction Price

## Scenario: Regression model for Taxi fares

Regression is a ML task type of supervised machine learning algorithms. 
A regression ML model predicts continuous value outputs (such as numbers). 
For instance, predicting the fare of a Taxi trip or predicting the price of a car is a regression problem.

In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML"

//Install XPlot package
#r "nuget:XPlot.Plotly"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.0

Installed package XPlot.Plotly version 3.0.1

In [2]:
using System;
using Microsoft.ML;
using System.IO;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;
using static Microsoft.ML.DataOperationsCatalog;
using XPlot.Plotly;

In [3]:
/// <summary>
/// The TaxiTrip class represents a single taxi trip.
/// </summary>
public class TaxiTrip
{
    [LoadColumn(0)] public string VendorId;
    [LoadColumn(5)] public string RateCode;
    [LoadColumn(3)] public float PassengerCount;
    [LoadColumn(4)] public float TripDistance;
    [LoadColumn(9)] public string PaymentType;
    [LoadColumn(10)] public float FareAmount;
}

/// <summary>
/// The TaxiTripFarePrediction class represents a single far prediction.
/// </summary>
public class TaxiTripFarePrediction
{
    [ColumnName("Score")] public float FareAmount;
}

In [4]:
// file paths to data files
static readonly string dataPath = Path.Combine(Environment.CurrentDirectory, "./Datasets/taxi/yellow_tripdata_2018-12.csv");

/// <summary>
/// The main application entry point.
/// </summary>
/// <param name="args">The command line arguments.</param>

// create the machine learning context
var mlContext = new MLContext();

// set up the text loader 
var textLoader = mlContext.Data.CreateTextLoader(
    new TextLoader.Options() 
    {
        Separators = new[] { ',' },
        HasHeader = true,
        Columns = new[] 
        {
            new TextLoader.Column("VendorId", DataKind.String, 0),
            new TextLoader.Column("RateCode", DataKind.String, 5),
            new TextLoader.Column("PassengerCount", DataKind.Single, 3),
            new TextLoader.Column("TripDistance", DataKind.Single, 4),
            new TextLoader.Column("PaymentType", DataKind.String, 9),
            new TextLoader.Column("FareAmount", DataKind.Single, 10)
        }
    }
);

// load the data 
Console.Write("Loading training data....");
var dataView = textLoader.Load(dataPath);
Console.WriteLine("done");

// split into a training and test partition
TrainTestData partitions  = mlContext.Data.TrainTestSplit(dataView, testFraction: 0.2);

Loading training data....done


In [5]:
display(h4("Schema of training DataView:"));
display(partitions.TrainSet.Schema);

Schema of training DataView:

index,Name,Index,IsHidden,Type,Annotations
0,VendorId,0,False,{ Microsoft.ML.Data.TextDataViewType: RawType: System.ReadOnlyMemory<System.Char> },{ Microsoft.ML.DataViewSchema+Annotations: Schema: [ ] }
1,RateCode,1,False,{ Microsoft.ML.Data.TextDataViewType: RawType: System.ReadOnlyMemory<System.Char> },{ Microsoft.ML.DataViewSchema+Annotations: Schema: [ ] }
2,PassengerCount,2,False,{ Microsoft.ML.Data.NumberDataViewType: RawType: System.Single },{ Microsoft.ML.DataViewSchema+Annotations: Schema: [ ] }
3,TripDistance,3,False,{ Microsoft.ML.Data.NumberDataViewType: RawType: System.Single },{ Microsoft.ML.DataViewSchema+Annotations: Schema: [ ] }
4,PaymentType,4,False,{ Microsoft.ML.Data.TextDataViewType: RawType: System.ReadOnlyMemory<System.Char> },{ Microsoft.ML.DataViewSchema+Annotations: Schema: [ ] }
5,FareAmount,5,False,{ Microsoft.ML.Data.NumberDataViewType: RawType: System.Single },{ Microsoft.ML.DataViewSchema+Annotations: Schema: [ ] }
6,SamplingKeyColumn,6,False,{ Microsoft.ML.Data.NumberDataViewType: RawType: System.Single },{ Microsoft.ML.DataViewSchema+Annotations: Schema: [ ] }


## Show a few rows of loaded data 

In [6]:
//Util class to preview loaded data in IDataView
public static List<TaxiTrip> Head(MLContext mlContext, IDataView dataView, int numberOfRows = 4)
{
    string msg = string.Format("DataView: Showing {0} rows with the columns", numberOfRows.ToString());
    display(msg);
          
    var rows = mlContext.Data.CreateEnumerable<TaxiTrip>(dataView, reuseRowObject: false)
                    .Take(numberOfRows)
                    .ToList();
    
    return rows;
}

display(h4("Showing a few rows from training DataView:"));

var fewRows = Head(mlContext, partitions.TrainSet, 5);
display(fewRows);

Showing a few rows from training DataView:

DataView: Showing 5 rows with the columns

index,VendorId,RateCode,PassengerCount,TripDistance,PaymentType,FareAmount
0,1,1,2,2.5,1,12
1,1,1,3,2.3,1,13
2,2,1,1,0,2,2.5
3,1,1,1,3.9,1,12.5
4,1,1,1,12.8,1,45


## Extract important input variables as arrays to be used for plotting

In [7]:
//Extract some data into arrays for plotting:

int numberOfRows = 1000;
float[] fares = partitions.TrainSet.GetColumn<float>("FareAmount").Take(numberOfRows).ToArray();
float[] distances = partitions.TrainSet.GetColumn<float>("TripDistance").Take(numberOfRows).ToArray();
float[] passengerCounts = partitions.TrainSet.GetColumn<float>("PassengerCount").Take(numberOfRows).ToArray();

## Show a histogram: Distribution of taxi trips per fare cost 

In [8]:
// Distribution of taxi trips per cost
//XPlot Histogram reference: http://tpetricek.github.io/XPlot/reference/xplot-plotly-graph-histogram.html

var faresHistogram = Chart.Plot(new Graph.Histogram(){x = fares, autobinx = false, nbinsx = 20});
var layout = new Layout.Layout(){title="Distribution of taxi trips per cost"};
faresHistogram.WithLayout(layout);
faresHistogram.WithXTitle("Fare ranges");
faresHistogram.WithYTitle("Number of trips");
display(faresHistogram);

# Plot Fares depending on trip's passengers 

In [9]:
// Plot Fare depending on Passengers

int numberOfRows = 2000;
float[] fares = partitions.TrainSet.GetColumn<float>("FareAmount").Take(numberOfRows).ToArray();
float[] passengerCounts = partitions.TrainSet.GetColumn<float>("PassengerCount").Take(numberOfRows).ToArray();

float[] distances = partitions.TrainSet.GetColumn<float>("TripDistance").Take(numberOfRows).ToArray();


var chartFareVsPassengers = Chart.Plot(
    new Graph.Scatter()
    {
        x = passengerCounts,
        y = fares,
        mode = "markers",
    }
);

var layout = new Layout.Layout(){title="Plot Fare depending on Passengers"};
chartFareVsPassengers.WithLayout(layout);
chartFareVsPassengers.Width = 500;
chartFareVsPassengers.Height = 500;
chartFareVsPassengers.WithXTitle("Passengers");
chartFareVsPassengers.WithYTitle("Fares");
chartFareVsPassengers.WithLegend(false);

display(chartFareVsPassengers);

In [10]:
display(h1("Apply Data Transformations pipeline"));
// set up a learning pipeline
var pipeline = mlContext.Transforms.CopyColumns(
    inputColumnName:"FareAmount", 
    outputColumnName:"Label")

    // one-hot encode all text features
    .Append(mlContext.Transforms.Categorical.OneHotEncoding("VendorId"))
    .Append(mlContext.Transforms.Categorical.OneHotEncoding("RateCode"))
    .Append(mlContext.Transforms.Categorical.OneHotEncoding("PaymentType"))

    // combine all input features into a single column 
    .Append(mlContext.Transforms.Concatenate(
        "Features", 
        "VendorId", 
        "RateCode", 
        "PassengerCount", 
        "TripDistance", 
        "PaymentType"))

    // cache the data to speed up training
    .AppendCacheCheckpoint(mlContext)

    // use the fast tree learner 
    .Append(mlContext.Regression.Trainers.OnlineGradientDescent(labelColumnName: "FareAmount", featureColumnName: "Features"));


// train the model
Console.Write("Training the model....");
var model = pipeline.Fit(partitions.TrainSet);
Console.WriteLine("done");

Apply Data Transformations pipeline

Training the model....done


In [11]:
// get a set of predictions 
Console.Write("Evaluating the model....");
var predictions = model.Transform(partitions.TestSet);


// get regression metrics to score the model
var metrics = mlContext.Regression.Evaluate(predictions, labelColumnName: "FareAmount", scoreColumnName: "Score");
display(metrics);

Evaluating the model....

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
2.6645709155534556,4091.869165164129,63.96771971208704,4091.8693689319734,0.027154999233267252


In [12]:
// create a prediction engine for one single prediction
var prediction  = mlContext.Model.CreatePredictionEngine<TaxiTrip, TaxiTripFarePrediction>(model).Predict(
    new TaxiTrip()
    {
        VendorId = "VTS",
        RateCode = "1",
        PassengerCount = 1,
        TripDistance = 3.75f,
        PaymentType = "1",
        FareAmount = 0 // actual fare for this trip = 15.5
    });

// show the prediction
Console.WriteLine($"Single prediction:");
Console.WriteLine($"  Predicted fare: {prediction.FareAmount:0.####}");
Console.WriteLine($"  Actual fare: 15.5");

Single prediction:
  Predicted fare: 11,2124
  Actual fare: 15.5
